In [1]:
!wget https://s3.amazonaws.com/text-datasets/imdb_full.pkl
!wget https://s3.amazonaws.com/text-datasets/imdb_word_index.json

--2022-06-09 06:53:49--  https://s3.amazonaws.com/text-datasets/imdb_full.pkl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.73.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.73.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65552540 (63M) [application/octet-stream]
Saving to: ‘imdb_full.pkl.2’

imdb_full.pkl.2     100%[===================>]  62.52M  23.1MB/s    in 2.7s    

2022-06-09 06:53:52 (23.1 MB/s) - ‘imdb_full.pkl.2’ saved [65552540/65552540]

--2022-06-09 06:53:52--  https://s3.amazonaws.com/text-datasets/imdb_word_index.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.73.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.73.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1641221 (1.6M) [application/octet-stream]
Saving to: ‘imdb_word_index.json.2’

imdb_word_index.jso 100%[===================>]   1.56M  --.-KB/s    in 0.06s   

2022-06-09 06:53:52 (26.5 MB/s) - ‘imdb_word_in

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import pickle
data = pickle.load(open('imdb_full.pkl', 'rb'))

import json
vocab = json.load(open('imdb_word_index.json'))

In [13]:
inv = {idx:word for word, idx in vocab.items()}

In [14]:
(X_train, y_train), (Xt, yt) = data

In [15]:
trainidx = [i for i, x in enumerate(X_train) if len(x) < 400]
trainidx, devidx = train_test_split(trainidx, train_size=0.8, random_state=1378)
X = [X_train[i] for i in trainidx]
y = [y_train[i] for i in trainidx]

Xd = [X_train[i] for i in devidx]
yd = [y_train[i] for i in devidx]

testidx = [i for i, x in enumerate(Xt) if len(x) < 400]
testidx, remaining_idx =  train_test_split(testidx, train_size=0.2, random_state=1378)

Xt = [Xt[i] for i in testidx]
yt = [yt[i] for i in testidx]

/home/yila22/anaconda3/envs/xai/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [16]:
def invert_and_join(X) :
    X = [[inv[x] for x in doc] for doc in X]
    X = [" ".join(x) for x in X]
    return X

In [17]:
X = invert_and_join(X)
Xd = invert_and_join(Xd)
Xt = invert_and_join(Xt)

In [18]:
texts = {'train' : X, 'test' : Xt, 'dev' : Xd}
labels = {'train' : y, 'test' : yt, 'dev' : yd}

In [9]:
import pandas as pd
df_texts = []
df_labels = []
df_exp_splits = []

for key in ['train', 'test', 'dev'] :
    df_texts += texts[key]
    df_labels += labels[key]
    df_exp_splits += [key] * len(texts[key])
    
df = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_splits})
df.to_csv('imdb_dataset.csv', index=False)

In [20]:
# !export PYTHONPATH=/home/yila22/prj
# !echo $PYTHONPATH
# %run "../preprocess_data_BC.py" --data_file imdb_dataset.csv --output_file ./vec_imdb.p --word_vectors_type fasttext.simple.300d --min_df 10

In [ ]:
cd ~/prj/attention/preprocess/IMDB
python ../preprocess_data_BC.py --data_file imdb_dataset.csv --output_file ./vec_imdb.p --word_vectors_type fasttext.simple.300d --min_df 10